# Loading Libraries

In [16]:
!pip install -q streamlit
!pip install -U "transformers==4.40.0" --upgrade
!pip install accelerate bitsandbytes
!npm install localtunnel
!pip install pix2text

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.759s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


In [17]:
import urllib


In [18]:
!pip install pix2text

In [19]:
!pip install onnxruntime-gpu


# Building the App

In [30]:
%%writefile app.py
import streamlit as st
from PIL import Image
import numpy as np
from pix2text import Pix2Text
import urllib
from io import BytesIO
import transformers
import torch
import accelerate
import bitsandbytes

p2t = Pix2Text.from_config()

model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
)

# Title
st.title("Solve your Problem Step by Step.")

# Description with website link
st.write("""
The model is built using Pix2Text and Llama3 with the help of Unsloth. Pix2Text converts the image into text and LaTeX code, while Llama3 with a prompt tries to solve your problem step by step.
Keep in mind that the model can sometimes generate incorrect answers, so it's advisable to verify the solutions. However, the model can provide useful insights into solving your problem.
The codes are organized by [Srijit Mukherjee](https://www.linkedin.com/in/srijit-mukherjee/), a Machine Learning researcher at Penn State.
""")

# Prompt for image upload
st.write("Upload your doubt problem in image format")

# Create a file uploader
uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    # Open the image using PIL
    img = Image.open(uploaded_file)

    # Run the image through the model
    math = p2t.recognize_text_formula(img)

    st.write("Problem:")
    st.write(math)

    messages = [
        {"role": "system", "content": '"""' + math + '"""'},
        {"role": "user", "content": """Solve this problem step by step in latex. Write the mathematics using the format of $ latex code $ for inline math."""},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=1200,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )

    output = outputs[0]["generated_text"][len(prompt):]
    #output = math

    # Display the output
    st.write("Step by Step Solution:")
    st.write(output)

# latext = r'''
# ## Latex example
# ### full equation
# $$
# \Delta G = \Delta\sigma \frac{a}{b}
# $$
# ### inline
# Assume $\frac{a}{b}=1$ and $\sigma=0$...
# '''
# st.write(latext)

Overwriting app.py


# Loading App

In [ ]:
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
!streamlit run /content/app.py &>/content/logs.txt &
!npx localtunnel --port 8501

Password/Enpoint IP for localtunnel is: 34.16.140.98
npx: installed 22 in 2.174s
your url is: https://three-clocks-unite.loca.lt
